<a href="https://colab.research.google.com/github/QSLV/personalized_fashion_recs/blob/main/feature_construct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install kaggle
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
!wget https://raw.githubusercontent.com/benhamner/Metrics/master/Python/ml_metrics/average_precision.py

--2022-04-25 01:51:47--  https://raw.githubusercontent.com/benhamner/Metrics/master/Python/ml_metrics/average_precision.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1654 (1.6K) [text/plain]
Saving to: ‘average_precision.py’

average_precision.p 100%[===================>]   1.62K  --.-KB/s    in 0s      

2022-04-25 01:51:47 (23.6 MB/s) - ‘average_precision.py’ saved [1654/1654]



In [2]:
# Load Packages
import pandas as pd
import numpy as np
from lightgbm.sklearn import LGBMRanker
from average_precision import apk, mapk
import random
from sklearn.base import BaseEstimator, TransformerMixin

## 读取数据

In [5]:
%%time
transactions = pd.read_parquet('/content/drive/MyDrive/H&M/data/transactions_train.parquet')
customers = pd.read_parquet('/content/drive/MyDrive/H&M/data/customers.parquet')
articles = pd.read_parquet('/content/drive/MyDrive/H&M/data/articles.parquet')

CPU times: user 2.1 s, sys: 1.81 s, total: 3.91 s
Wall time: 2.99 s


In [6]:
display(transactions.head())
display(customers.head())
display(articles.head())


,t_dat,customer_id,article_id,price,sales_channel_id,week
25784,2018-09-20,1728846800780188,519773001,0.028458,2,0
25785,2018-09-20,1728846800780188,578472001,0.032525,2,0
5389,2018-09-20,2076973761519164,661795002,0.167797,2,0
5390,2018-09-20,2076973761519164,684080003,0.101678,2,0
47429,2018-09-20,2918879973994241,662980001,0.033881,1,0


,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,6883939031699146327,-1,-1,0,0,49,6305
1,11246327431398957306,-1,-1,0,0,25,33726
2,18439897732908966680,-1,-1,0,0,24,3247
3,18352672461570950206,-1,-1,0,0,54,168643
4,18162778555210377306,1,1,0,1,52,168645


,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,12855,253,9,0,1010016,0,9,0,...,10,0,0,1,0,16,30,1002,2,8834
1,108775044,108775,12855,253,9,0,1010016,0,10,2,...,10,0,0,1,0,16,30,1002,2,8834
2,108775051,108775,44846,253,9,0,1010017,3,11,11,...,10,0,0,1,0,16,30,1002,2,8834
3,110065001,110065,8159,306,13,4,1010016,0,9,0,...,131,7,7,1,0,61,5,1017,4,8243
4,110065002,110065,8159,306,13,4,1010016,0,10,2,...,131,7,7,1,0,61,5,1017,4,8243


In [7]:
articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105542 entries, 0 to 105541
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype
---  ------                        --------------   -----
 0   article_id                    105542 non-null  int32
 1   product_code                  105542 non-null  int32
 2   prod_name                     105542 non-null  int32
 3   product_type_no               105542 non-null  int32
 4   product_type_name             105542 non-null  int16
 5   product_group_name            105542 non-null  int8 
 6   graphical_appearance_no       105542 non-null  int32
 7   graphical_appearance_name     105542 non-null  int8 
 8   colour_group_code             105542 non-null  int32
 9   colour_group_name             105542 non-null  int8 
 10  perceived_colour_value_id     105542 non-null  int32
 11  perceived_colour_value_name   105542 non-null  int8 
 12  perceived_colour_master_id    105542 non-null  int32
 13  perceived_colo

In [15]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31788324 entries, 25784 to 31780475
Data columns (total 6 columns):
 #   Column            Dtype         
---  ------            -----         
 0   t_dat             datetime64[ns]
 1   customer_id       uint64        
 2   article_id        int32         
 3   price             float32       
 4   sales_channel_id  int8          
 5   week              int8          
dtypes: datetime64[ns](1), float32(1), int32(1), int8(2), uint64(1)
memory usage: 1.0 GB


In [9]:
all_df = pd.merge(transactions, articles, on='article_id', how='left')
all_df = pd.merge(all_df, customers, on='customer_id', how='left')


In [26]:
all_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31788324 entries, 0 to 31788323
Data columns (total 36 columns):
 #   Column                        Dtype         
---  ------                        -----         
 0   t_dat                         datetime64[ns]
 1   customer_id                   uint64        
 2   article_id                    int32         
 3   price                         float32       
 4   sales_channel_id              int8          
 5   week                          int8          
 6   product_code                  int32         
 7   prod_name                     int32         
 8   product_type_no               int32         
 9   product_type_name             int16         
 10  product_group_name            int8          
 11  graphical_appearance_no       int32         
 12  graphical_appearance_name     int8          
 13  colour_group_code             int32         
 14  colour_group_name             int8          
 15  perceived_colour_value_id     

# 特征构建

In [11]:
def group_func(df, group_func_dic, group_key):
  if isinstance(group_func_dic, str):
    group_func_dic = [group_func_dic]
  features = df.groupby(group_key).agg(group_func_dic)
  features.columns = [e[0] + "_" + e[1].upper() for e in features.columns.tolist()]
  features.reset_index(inplace=True)
  return features
  
def add_prefix(df, exclude_columns, prefix):
    if isinstance(exclude_columns, str):
        exclude_columns = [exclude_columns]
        
    column_names = [col for col in df.columns if col not in exclude_columns]
    df.rename(columns = dict(zip(column_names, [prefix + name for name in column_names])), inplace=True)
    return df

## 用户侧

In [18]:
# 用户基础统计特征
def get_user_feature(df):
  """
  用户基础统计特征：
    1. 下单数
    2. 购买渠道数量
    3. 行为时间： 按天、周 ， 去重数， 首次， 末次， 首末差
    4. 品类数
    5. 商品数量
    6. 商品价格： 最大 最小 平均
  """
  feature_type = {
      'customer_id': ['count'],
      'sales_channel_id': ['nunique'],
      't_dat': ['nunique', 'max', 'min', np.ptp],
      'week': ['nunique', 'max', 'min', np.ptp],
      'product_type_name': ['nunique'],
      'article_id': ['nunique'],
      'price': ['max', 'min', 'mean']
  }
  feature = group_func(df, feature_type, group_key=['customer_id'])
  feature = add_prefix(feature, ['customer_id'], 'customer_')
  return feature

In [19]:
%%time
user_feature = get_user_feature(all_df)
user_feature.head(10)

,customer_id,customer_customer_id_COUNT,customer_sales_channel_id_NUNIQUE,customer_t_dat_NUNIQUE,customer_t_dat_MAX,customer_t_dat_MIN,customer_t_dat_PTP,customer_week_NUNIQUE,customer_week_MAX,customer_week_MIN,customer_week_PTP,customer_product_type_name_NUNIQUE,customer_article_id_NUNIQUE,customer_price_MAX,customer_price_MIN,customer_price_MEAN
0,4245900472157,2,1,1,2020-04-10,2020-04-10,0 days,1,81,81,0,2,2,0.025407,0.016932,0.021169
1,23962613628581,20,2,2,2019-08-03,2019-07-18,16 days,2,45,43,2,5,18,0.067780,0.006763,0.036932
2,25398598941468,1,1,1,2019-11-18,2019-11-18,0 days,1,60,60,0,1,1,0.042356,0.042356,0.042356
3,28847241659200,74,2,45,2020-09-03,2018-09-30,704 days,37,102,1,101,23,71,0.128797,0.004051,0.033151
4,41046458195168,4,1,1,2020-04-02,2020-04-02,0 days,1,80,80,0,2,2,0.016932,0.016932,0.016932
5,41318098387474,32,2,13,2020-08-08,2018-10-18,660 days,13,98,4,94,10,31,0.033881,0.008458,0.019369
6,43757848922470,16,2,6,2019-07-12,2018-10-08,277 days,6,42,2,40,12,16,0.118627,0.005068,0.029212
7,46878247658203,1,1,1,2020-04-17,2020-04-17,0 days,1,82,82,0,1,1,0.016932,0.016932,0.016932
8,56938741737809,12,2,3,2020-01-09,2019-02-22,321 days,3,68,22,46,9,12,0.036000,0.003373,0.014284
9,77117344919861,294,2,56,2020-06-15,2018-09-21,633 days,45,90,0,90,38,216,0.118627,0.005746,0.034021


## 商品侧特征

In [32]:
# 商品品类特征 - 粗粒度 product_group_name
def get_product_group_feature(df):
  """
  品类基础特征：
  1. 行为数
  2. 行为时间：去重，首次，末次，首末差
      i: 天
      iii: 周
  3. 用户数：去重
  4. 商品数：去重
  5. 细类数量
  6. 商品价格：最大、最小、差值
  """
  feature_type = {
      'product_group_name': ['count'],
      't_dat': ['nunique', 'max', 'min', np.ptp],
      'week': ['nunique', 'max', 'min', np.ptp],
      'customer_id': ['nunique'],
      'article_id': ['nunique'],
      'product_type_name': ['nunique'],
      'price': ['max', 'min', 'mean', np.ptp],
  }
  feature = group_func(df, feature_type, group_key=['product_group_name'])
  feature = add_prefix(feature, ['product_group_name'], 'product_group_name_')
  return feature



In [25]:
%%time
product_group_feature = get_product_group_feature(all_df)

CPU times: user 38.5 s, sys: 99.7 ms, total: 38.6 s
Wall time: 38.4 s


In [29]:
product_group_feature

,product_group_name,product_group_name_product_group_name_COUNT,product_group_name_t_dat_NUNIQUE,product_group_name_t_dat_MAX,product_group_name_t_dat_MIN,product_group_name_t_dat_PTP,product_group_name_week_NUNIQUE,product_group_name_week_MAX,product_group_name_week_MIN,product_group_name_week_PTP,product_group_name_customer_id_NUNIQUE,product_group_name_article_id_NUNIQUE,product_group_name_product_type_name_NUNIQUE,product_group_name_price_MAX,product_group_name_price_MIN,product_group_name_price_MEAN,product_group_name_price_PTP
0,0,12552755,734,2020-09-22,2018-09-20,733 days,105,104,0,104,1092325,42313,15,0.591525,0.000017,0.027005,0.591509
1,1,7046054,734,2020-09-22,2018-09-20,733 days,105,104,0,104,927480,19661,5,0.422034,0.000186,0.032894,0.421847
2,2,3552470,734,2020-09-22,2018-09-20,733 days,105,104,0,104,644925,13160,6,0.506780,0.000407,0.036166,0.506373
3,3,1599593,734,2020-09-22,2018-09-20,733 days,105,104,0,104,491251,11023,38,0.337288,0.000034,0.015563,0.337254
4,4,2565858,734,2020-09-22,2018-09-20,733 days,105,104,0,104,517715,5447,11,0.118627,0.000051,0.021200,0.118576
5,5,745521,734,2020-09-22,2018-09-20,733 days,105,104,0,104,269195,5228,15,0.337288,0.000203,0.038749,0.337085
6,6,2579222,734,2020-09-22,2018-09-20,733 days,105,104,0,104,426146,3127,6,0.084729,0.000119,0.022343,0.084610
7,7,685712,734,2020-09-22,2018-09-20,733 days,105,104,0,104,267096,2420,3,0.098288,0.000119,0.011391,0.098169
8,8,348180,734,2020-09-22,2018-09-20,733 days,105,104,0,104,169597,1882,4,0.084729,0.000831,0.025426,0.083898
9,9,97040,734,2020-09-22,2018-09-20,733 days,105,104,0,104,64345,113,1,0.101678,0.001000,0.026780,0.100678


In [31]:
# 商品品类特征 - 细粒度 product_type_name
def get_product_type_feature(df):
  """
  品类基础特征：
  1. 行为数
  2. 行为时间：去重，首次，末次，首末差
      i: 天
      iii: 周
  3. 用户数：去重
  4. 商品数：去重
  5. 商品价格：最大、最小、差值
  """
  feature_type = {
      'product_type_name': ['count'],
      't_dat': ['nunique', 'max', 'min', np.ptp],
      'week': ['nunique', 'max', 'min', np.ptp],
      'customer_id': ['nunique'],
      'article_id': ['nunique'],
      'price': ['max', 'min', 'mean', np.ptp],
  }
  feature = group_func(df, feature_type, group_key=['product_type_name'])
  feature = add_prefix(feature, ['product_type_name'], 'product_type_name_')
  return feature



In [33]:
%%time
product_type_feature = get_product_type_feature(all_df)

CPU times: user 40.3 s, sys: 92.7 ms, total: 40.4 s
Wall time: 40.2 s


In [34]:
product_type_feature.head(10)

,product_type_name,product_type_name_product_type_name_COUNT,product_type_name_t_dat_NUNIQUE,product_type_name_t_dat_MAX,product_type_name_t_dat_MIN,product_type_name_t_dat_PTP,product_type_name_week_NUNIQUE,product_type_name_week_MAX,product_type_name_week_MIN,product_type_name_week_PTP,product_type_name_customer_id_NUNIQUE,product_type_name_article_id_NUNIQUE,product_type_name_price_MAX,product_type_name_price_MIN,product_type_name_price_MEAN,product_type_name_price_PTP
0,0,4217017,734,2020-09-22,2018-09-20,733 days,105,104,0,104,738844,11060,0.422034,0.000356,0.035785,0.421678
1,1,3238428,734,2020-09-22,2018-09-20,733 days,105,104,0,104,608936,10266,0.506780,0.000407,0.036495,0.506373
2,2,2783274,734,2020-09-22,2018-09-20,733 days,105,104,0,104,642456,9173,0.252542,0.000322,0.029006,0.252220
3,3,2203750,734,2020-09-22,2018-09-20,733 days,105,104,0,104,558353,7873,0.167797,0.000322,0.013738,0.167475
4,4,1583408,734,2020-09-22,2018-09-20,733 days,105,104,0,104,506862,4104,0.252542,0.000203,0.020286,0.252339
5,5,1504868,734,2020-09-22,2018-09-20,733 days,105,104,0,104,441263,3941,0.422034,0.000017,0.028192,0.422017
6,6,600157,734,2020-09-22,2018-09-20,733 days,105,104,0,104,279169,3865,0.506780,0.001475,0.067354,0.505305
7,7,1152513,734,2020-09-22,2018-09-20,733 days,105,104,0,104,358567,3937,0.135576,0.000288,0.023404,0.135288
8,8,787894,734,2020-09-22,2018-09-20,733 days,105,104,0,104,329358,3391,0.422034,0.000559,0.026936,0.421475
9,9,1414101,734,2020-09-22,2018-09-20,733 days,105,104,0,104,424945,2987,0.118627,0.000203,0.015658,0.118424


In [39]:
# 商品特征 
def get_article_feature(df):
  """
  品类基础特征：
  1. 行为数
  2. 行为时间：去重，首次，末次，首末差
      i: 天
      iii: 周
  3. 用户数：去重
  4. 品类数：细粒度， 粗粒度 去重
  5. 商品价格：最大、最小、平均 std 差值
  """
  feature_type = {
      'article_id': ['count'],
      't_dat': ['nunique', 'max', 'min', np.ptp],
      'week': ['nunique', 'max', 'min', np.ptp],
      'customer_id': ['nunique'],
      'product_type_name': ['nunique'],
      'product_group_name': ['nunique'],
      'price': ['max', 'min', 'mean', np.ptp, 'std'],
  }
  feature = group_func(df, feature_type, group_key=['article_id'])
  feature = add_prefix(feature, ['article_id'], 'article_id_')
  return feature



In [40]:
%%time
articles_feature = get_article_feature(all_df)
display(articles_feature.head(10))

,article_id,article_id_article_id_COUNT,article_id_t_dat_NUNIQUE,article_id_t_dat_MAX,article_id_t_dat_MIN,article_id_t_dat_PTP,article_id_week_NUNIQUE,article_id_week_MAX,article_id_week_MIN,article_id_week_PTP,article_id_customer_id_NUNIQUE,article_id_product_type_name_NUNIQUE,article_id_product_group_name_NUNIQUE,article_id_price_MAX,article_id_price_MIN,article_id_price_MEAN,article_id_price_PTP,article_id_price_STD
0,108775015,10841,385,2020-07-22,2018-09-20,671 days,79,96,0,96,6885,1,1,0.009153,0.001339,0.008142,0.007814,0.000652
1,108775044,7250,516,2020-09-20,2018-09-20,731 days,105,104,0,104,5284,1,1,0.008508,0.001424,0.008114,0.007085,0.000735
2,108775051,215,56,2019-06-28,2018-09-20,281 days,16,40,0,40,168,1,1,0.008458,0.003237,0.004980,0.005220,0.000544
3,110065001,1044,382,2020-08-02,2018-09-20,682 days,86,97,0,97,945,1,1,0.025407,0.002525,0.020219,0.022881,0.006844
4,110065002,539,245,2020-08-05,2018-09-20,685 days,75,98,0,98,460,1,1,0.025407,0.004542,0.018205,0.020864,0.006004
5,110065011,1090,345,2020-09-01,2018-09-20,712 days,81,101,0,101,968,1,1,0.025407,0.003373,0.018510,0.022034,0.006340
6,111565001,4911,679,2020-09-22,2018-09-20,733 days,104,104,0,104,3585,1,1,0.008458,0.000322,0.006904,0.008136,0.001854
7,111565003,46,31,2019-02-05,2018-10-31,97 days,13,19,6,13,41,1,1,0.008458,0.006763,0.008274,0.001695,0.000485
8,111586001,14191,629,2020-09-22,2018-09-20,733 days,99,104,0,104,7903,1,1,0.016983,0.001085,0.012036,0.015898,0.002474
9,111593001,13888,663,2020-09-22,2018-09-20,733 days,99,104,0,104,7647,1,1,0.016932,0.002814,0.012185,0.014119,0.002548


CPU times: user 1min 31s, sys: 459 ms, total: 1min 31s
Wall time: 1min 30s
